In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split



In [19]:

movieReviews = pd.read_json('IMDB_reviews.json', lines=True)

# convert the date column into a datetime object
movieReviews['review_date'] = pd.to_datetime(movieReviews['review_date'])
movieReviews['day'] = movieReviews['review_date'].dt.day
movieReviews['month'] = movieReviews['review_date'].dt.month
movieReviews['year'] = movieReviews['review_date'].dt.year
movieReviews['review_combined'] = movieReviews['review_summary']+ ' ' + movieReviews['review_text']
movieReviews = movieReviews.drop(columns=['review_summary','review_text', 'review_date'])


In [22]:
movieReviews

,movie_id,user_id,is_spoiler,rating,day,month,year,review_combined
0,tt0111161,ur1898687,True,10,10,2,2006,A classic piece of unforgettable film-making. ...
1,tt0111161,ur0842118,True,10,6,9,2000,Simply amazing. The best film of the 90's. The...
2,tt0111161,ur1285640,True,8,3,8,2001,The best story ever told on film I believe tha...
3,tt0111161,ur1003471,True,10,1,9,2002,"Busy dying or busy living? **Yes, there are SP..."
4,tt0111161,ur0226855,True,8,20,5,2004,"Great story, wondrously told and acted At the ..."
...,...,...,...,...,...,...,...,...
573908,tt0139239,ur0100166,False,10,8,8,1999,The best teen movie of the nineties Go is wise...
573909,tt0139239,ur0021767,False,9,31,7,1999,"Go - see the movie Well, what shall I say. thi..."
573910,tt0139239,ur0392750,False,10,20,7,1999,It's the best movie I've ever seen Go is the b...
573911,tt0139239,ur0349105,False,3,11,6,1999,Haven't we seen this before? Call this 1999 te...


In [25]:
review = movieReviews[movieReviews['year']==2017]
unused_columns = ['day','user_id','movie_id','month','rating','year']
review  = review.drop(unused_columns,axis=1)
review

,is_spoiler,review_combined
75,True,Awesome Classic This movie will be one of my f...
76,True,"If you a movie lover, Don't miss a second to s..."
77,True,Mona Lisa of Film We just do not get movies li...
78,True,"Very nice! Excellent movie, I have watched it ..."
79,True,Worthy of the number one spot. One of the best...
...,...,...
573265,False,Teens set out in the woods of Burkittsville to...
573283,False,Annoying screaming and crying for 80 minutes A...
573284,False,A Disappointment People will say the movie is ...
573430,False,One of the most effective horror movies ever m...


In [26]:
import unidecode
import re
import time
import string
import statistics
#import contractions

# Remove links from text
def remove_links(text):
    text = re.sub(r'http\S+', '', text)
    return text

# Remove accented characters
def accented_characters_removal(text):
    text = unidecode.unidecode(text)
    return text

# Remove repeated characters and punctuations
def reducing_incorrect_character_repeatation(text):
    # Pattern matching for all case alphabets
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)

    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text)

    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')

    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)

    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)

    return Final_Formatted

# Expand contraction words
#def expand_contractions(text):
#    expanded_words = []
#    for word in text.split():
#      expanded_words.append(contractions.fix(word))
#    String_Of_tokens = ' '.join(expanded_words)
#    return String_Of_tokens

# Remove special characters
def removing_special_characters(text):
    pattern = r"[^a-zA-Z]"
    text = re.sub(pattern,' ', text)
    return text

# Text cleaning
def text_clean(text):
    text = text.lower()
    text = remove_links(text)
    #text = expand_contractions(text)
    text = accented_characters_removal(text)
    text = removing_special_characters(text)
    text = reducing_incorrect_character_repeatation(text)
    return text

review['review_combined'] = review['review_combined'].apply(text_clean)

In [28]:
# from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
from nltk.corpus import stopwords # Stopwords
from nltk.tokenize import word_tokenize # Word_tokenizer
import re
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import cross_validate

import nltk
from nltk.corpus import names

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('names')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bingy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bingy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\bingy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\names.zip.


,is_spoiler,review_combined,tokenized
75,True,awesome classic this movie will be one of my f...,"[awesom, classic, thi, movi, one, favorit, tim..."
76,True,if you a movie lover don t miss a second to se...,"[movi, lover, miss, second, see, thi, movi, sh..."
77,True,mona lisa of film we just do not get movies li...,"[mona, lisa, film, get, movi, like, anymor, sh..."
78,True,very nice excellent movie i have watched it tw...,"[veri, nice, excel, movi, watch, twice, thrice..."
79,True,worthy of the number one spot one of the best ...,"[worthi, number, one, spot, one, best, movi, d..."


In [29]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def tokenizer(entry):
    final_lst = []
    lst = word_tokenize(entry)
    lst_updated = [stemmer.stem(word) for word in lst]
    for i in lst_updated:
        if i not in stop_words:
            final_lst.append(i)

    return final_lst

def tokenize_series(lst):
    final_lst = []
    for entry in lst:
        tokenized_entry = tokenizer(entry)
        final_lst.append(tokenized_entry)
    return final_lst

review['tokenized'] = tokenize_series(review['review_combined'])
review.head()

,is_spoiler,review_combined,tokenized
75,True,awesome classic this movie will be one of my f...,"[awesom, classic, thi, movi, one, favorit, tim..."
76,True,if you a movie lover don t miss a second to se...,"[movi, lover, miss, second, see, thi, movi, sh..."
77,True,mona lisa of film we just do not get movies li...,"[mona, lisa, film, get, movi, like, anymor, sh..."
78,True,very nice excellent movie i have watched it tw...,"[veri, nice, excel, movi, watch, twice, thrice..."
79,True,worthy of the number one spot one of the best ...,"[worthi, number, one, spot, one, best, movi, d..."


In [31]:
review.to_json('review_in_2017.json')

In [4]:

#tokenized.json was done by Anji, preserving unused_columns
tokenized = pd.read_json('tokenized.json')
tokenized.head()

,user_id,is_spoiler,rating_x,tokenized,duration,genre,release_date
0,ur81689327,True,9,"[awesom, classic, thi, movi, one, favorit, tim...",2,"[Crime, Drama]",1994
1,ur70090039,True,10,"[movi, lover, miss, second, protagonist, thi, ...",2,"[Crime, Drama]",1994
2,ur79297787,True,9,"[protagonist, protagonist, film, get, movi, li...",2,"[Crime, Drama]",1994
3,ur79209549,True,10,"[veri, nice, excel, movi, watch, twice, thrice...",2,"[Crime, Drama]",1994
4,ur20574753,True,10,"[worthi, number, one, spot, one, best, movi, d...",2,"[Crime, Drama]",1994
